In [1]:
!pip install --upgrade transformers==4.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 132.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 277.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.3/773.3 kB 345.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 251.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 326.8 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=bbacbea935540b174dfae5e5a744086a01b96a08581ec3c984f49b1f2a68eae4
  Stored in directory: /tmp/pip-ephem-wheel-cache-zxicoapw/wheels/12/1c/3d/46cf06718d63a32ff798a89594b61e7f345ab6b36d909ce033
Successfully built sacremoses


In [2]:
!pip install torch tensorflow flax

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 263.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.8/489.8 MB 267.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 315.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 256.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 243.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.3/89.3 MB 214.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 366.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 301.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.4/173.4 kB 297.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 294.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 268.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import numpy as np
from collections import defaultdict
from typing import List, Tuple

import pandas as pd

from transformers import AutoTokenizer, AutoModel

Build a vector database

In [4]:
class VectorDatabase:
    def __init__(self):
        self.vectors = defaultdict(np.ndarray)

    def insert(self, key: str, vector: np.ndarray) -> None:
        self.vectors[key] = vector

    def search(self, query_vector: np.ndarray, k: int) -> List[Tuple[str, float]]:
        similarities = [(key, cosine(query_vector, vector)) for key, vector in self.vectors.items()]
        similarities.sort(key=lambda x: x[1], reverse=True)
        return similarities[:k]

    def retrieve(self, key: str) -> np.ndarray:
        return self.vectors.get(key, None)

In [5]:
def cosine(v1: np.ndarray, v2: np.ndarray) -> float:
    result = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
    return result

In [6]:
# get embeddings from the file
df = pd.read_csv("cord_19_embeddings_2022-06-02_test.csv", header=None)

In [42]:
# create a vector database
vector_db = VectorDatabase()

Embed the query

In [9]:
# load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('allenai/specter')
model = AutoModel.from_pretrained('allenai/specter')

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/222k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/321 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [23]:
papers = [
    {'title': 'BERT', 'abstract': 'We introduce a new language representation model called BERT'},
    {'title': 'Attention is all you need', 'abstract': 'The dominant sequence transduction models are based on complex recurrent or convolutional neural networks'},
    {'title': 'ImageNet Classification with Deep Convolutional Neural Networks', 'abstract': 'We trained a large, deep convolutional neural network to classify the 1.2 million high-resolution images in the ImageNet LSVRC-2010 contest.'},
    {'title': 'Mastering Chess and Shogi by Self-Play with a General Reinforcement Learning Algorithm', 'abstract': 'The game of chess is the most widely-studied domain in the history of artificial intelligence. We introduce an algorithm based purely on reinforcement learning, without human data.'},
    {'title': 'Playing Atari with Deep Reinforcement Learning', 'abstract': 'We present the first deep learning model to successfully learn control policies directly from high-dimensional sensory input using reinforcement learning.'},
    {'title': 'Gradient-based learning applied to document recognition', 'abstract': 'Procedures for training the multi-layer perceptron are discussed in a comprehensive framework.'},
    {'title': 'Understanding Machine Learning: From Theory to Algorithms', 'abstract': 'This book provides a theoretical account of the fundamentals underlying machine learning and the mathematical derivations that transform these principles into practical algorithms.'},
    {'title': 'A Few Useful Things to Know About Machine Learning', 'abstract': 'This paper provides a general, high-level overview of important concepts in machine learning.'},
    {'title': 'Dropout: A Simple Way to Prevent Neural Networks from Overfitting', 'abstract': 'Dropout is a technique for addressing the overfitting problem in large neural networks.'},
    {'title': 'Sequence to Sequence Learning with Neural Networks', 'abstract': 'We propose a deep learning approach for translating short sentences using neural networks.'},
    {'title': 'Word2Vec', 'abstract': 'We propose two novel model architectures for computing continuous vector representations of words from very large data sets.'},
    {'title': 'Zero-shot learning with semantic output codes', 'abstract': 'Zero-shot learning is a paradigm which bypasses the need for labeled examples for each possible class.'},
    {'title': 'Neural Machine Translation by Jointly Learning to Align and Translate', 'abstract': 'We propose a novel neural network architecture for machine translation that models the alignment between source and target sequences.'},
    {'title': 'Deep Residual Learning for Image Recognition', 'abstract': 'Deeper neural networks are more difficult to train. We present a residual learning framework to ease the training of networks that are substantially deeper than those used previously.'},
    {'title': 'Generative Adversarial Networks', 'abstract': 'We propose a new framework for estimating generative models via an adversarial process.'},
    {'title': 'FaceNet: A Unified Embedding for Face Recognition and Clustering', 'abstract': 'We present a system, called FaceNet, that directly learns a mapping from face images to a compact Euclidean space.'},
    {'title': 'YOLO: You Only Look Once', 'abstract': 'We present YOLO, a new approach to object detection. Our model makes a single look at an image and predicts what objects are present and where they are.'},
    {'title': 'Style Transfer', 'abstract': 'We present an approach to rapidly and easily generate a wide variety of image styles from a single input image.'},
    {'title': 'Graph Neural Networks', 'abstract': 'We propose a novel type of neural network model for processing structured data with graph.'},
    {'title': 'The Transformer Family: A Survey', 'abstract': 'This paper surveys the various models that have been introduced based on the Transformer architecture and its applications.'},
    
    {'title': 'Molecular Structure of Nucleic Acids', 'abstract': 'A structure for deoxyribose nucleic acid (DNA). Watson and Crick propose the double helix model of DNA.'},
    {'title': 'The Sequence of the Human Genome', 'abstract': 'An initial draft and analysis of the human genome, revealing the vast amount of information encoded within our DNA.'},
    {'title': 'Genome sequence of the nematode C. elegans: A platform for investigating biology', 'abstract': 'The complete genome sequence of the nematode Caenorhabditis elegans offers insights into animal biology and evolution.'},
    {'title': 'RNA interference: traveling in the cell and gaining functions?', 'abstract': 'A review of RNA interference (RNAi) mechanisms, its cellular applications, and functional gains.'},
    {'title': 'DNA methylation and gene function', 'abstract': 'Exploring the relationship between DNA methylation patterns and gene function.'},
    {'title': 'The Hallmarks of Cancer', 'abstract': 'The complexities of cancer biology are distilled into a set of underlying principles, shedding light on cancer biological properties.'},
    {'title': 'Splicing in the Inner Ear: A Familiar Tune, but What Are the Instruments?', 'abstract': 'Insights into alternative splicing events in the inner ear.'},
    {'title': 'Principles of microRNA–target recognition', 'abstract': 'Exploring the principles governing the recognition of target mRNAs by miRNAs.'},
    {'title': 'The Polymerase Chain Reaction', 'abstract': 'Introduction and exploration of the PCR technique, a foundational method for amplifying DNA.'},
    {'title': 'Targeting of the CREB gene leads to up-regulation of a novel CREB mRNA isoform', 'abstract': 'Insights into gene targeting and the role of CREB in transcription.'},
    {'title': 'The Unfolded Protein Response: From Stress Pathway to Homeostatic Regulation', 'abstract': 'Understanding the UPR and its role in maintaining cellular homeostasis.'},
    {'title': 'Mechanisms and Functions of DNA Mismatch Repair', 'abstract': 'A comprehensive review of DNA mismatch repair and its implications in genomic stability.'},
    {'title': 'The Genomic Basis of Adaptive Evolution in Threespine Sticklebacks', 'abstract': 'Exploring the genomic changes accompanying adaptation in stickleback fish.'},
    {'title': 'MicroRNAs: genomics, biogenesis, mechanism, and function', 'abstract': 'A comprehensive review of microRNAs, their genesis, and function in gene regulation.'},
    {'title': 'Clonal expansion of mutated mitochondrial DNA is associated with tumor formation and complex I deficiency in the benign renal oncocytoma', 'abstract': 'Unraveling the role of mitochondrial DNA mutations in benign renal oncocytoma.'},
    {'title': 'The Structure of the Potassium Channel: Molecular Basis of K+ Conduction and Selectivity', 'abstract': 'Detailed insights into the structure and function of potassium ion channels.'},
    {'title': 'Telomeres and Telomerase: The Path from Maize, Tetrahymena and Yeast to Human Cancer and Aging', 'abstract': 'Understanding the role and function of telomeres and telomerase in aging and cancer.'},
    {'title': 'Tracing the origins of functional and conserved domains in the human genome: implications for protein evolution', 'abstract': 'Investigating the evolutionary origins of functional domains in human proteins.'},
    {'title': 'Transcriptional regulatory code of a eukaryotic genome', 'abstract': 'Deciphering the regulatory code governing transcription in eukaryotes.'},
    {'title': 'The Tumor Microenvironment and Immune Milieu of Cholangiocarcinoma', 'abstract': 'Understanding the complex interplay between tumors and the immune system in cholangiocarcinoma.'}
]

In [24]:
# concatenate title and abstract
title_abs = [d['title'] + tokenizer.sep_token + (d.get('abstract') or '') for d in papers]
# preprocess the input
inputs = tokenizer(title_abs, padding=True, truncation=True, return_tensors="pt", max_length=512)
result = model(**inputs)
# take the first token in the batch as the embedding
embeddings = result.last_hidden_state[:, 0, :]

In [26]:
embeddings.shape

torch.Size([40, 768])

In [45]:
# CURRENTLY IT WILL INSERT INTO THE ALREADY EXISTING DATABASE, BUT IT SHOULDN'T MATTER
keys = np.random.randint(1000, size=40)
# insert embeddings into the database
for i, row in enumerate(embeddings):
    key = keys[i]
    embedding = embeddings[i]
    embedding = embedding.detach().numpy()    
    vector_db.insert(key, embedding)

In [46]:
query = "The Hallmarks of Cancer"
inputs = tokenizer(query, padding=True, truncation=True, return_tensors="pt", max_length=512)
result = model(**inputs)
query_embedding = result.last_hidden_state[:, 0, :]
query_embedding = query_embedding.detach().numpy()

In [48]:
%%time
result = vector_db.search(query_embedding, k=5)
result # result will give the key for the best matched paper

CPU times: user 3.01 ms, sys: 0 ns, total: 3.01 ms
Wall time: 2.27 ms


[(664, array([0.9816665], dtype=float32)),
 (487, array([0.86776626], dtype=float32)),
 (496, array([0.8626136], dtype=float32)),
 (638, array([0.8496898], dtype=float32)),
 (627, array([0.8356982], dtype=float32))]

In [60]:
best_result = result[1][0]

In [61]:
index_in_papers = list(keys).index(best_result) # this will give the index of that paper in the papers array

In [62]:
papers[index_in_papers] # now find out the title and abstract of the found paper

{'title': 'Understanding Machine Learning: From Theory to Algorithms',
 'abstract': 'This book provides a theoretical account of the fundamentals underlying machine learning and the mathematical derivations that transform these principles into practical algorithms.'}